# Modèles génératifs markoviens

Dans ce TP, nous allons étudier la possibilité de générer du texte en langage naturel via une modélisation statistique.

Les langages naturels (le français, l'anglais etc) sont à la fois fortement structurés et très riches :
* On peut écrire une infinité de phrases correctes différentes
* L'immense majorité des suites de mots ne forme pas des phrases

La structuration grammaticale du langage entraîne que la distribution de fréquence marginale d'un mot est très différente de celle conditionnée par son contexte (*i.e.*, étant donnés les mots qui précèdent) : 

$$ P(X_i) \neq P(X_i|X_{i-1},\dots,X_1) $$

Or on peut raisonnablement considérer qu'un page particulière d'un romab particulier, qui contient entre 250 et 280 mots, identifie de manière unique le roman en question. Cela implique que le mot qui suit immédiatement cette page est determiné de manière unique dans l'ensemble de la production litteraire de l'humanité. Il en résulte que pour une page originale grammaticalement correcte, aucune donnée statistique n'est disponible pour determiner la probabilité qu'un mot particulier ne vienne ensuite. 

Notre rêve de pouvoir simplement modéliser la distribution de probabilité jointe $P(X_1,\dots,X_100000)$ de tous les livres de 100000 mots possibles et de simplement en tirer un au hasard s'évanouit définitivement...

Notons également qu'une telle distribution possède $n^100000$ dimensions où $n$ est le nombre de mots valides de la langue. Notre cause est donc désespérée

Nous allons donc devoir considérer un **contexte limité**. 

**Définition - Chaîne de Markov:** Une chaîne de Markov est une suite de variables aléatoires $(X_i)_i$ telle que 

$$ P(X_i+1|X_i) = P(X_i+1|X_i,\dots,X_1) $$

Autrement dit, la distribution de la prochaine variable ne dépend que de la variable actuelle. Une telle chaîne de Markov est dite "sans mémoire", car elle "oublie" le contexte antérieur.

La génération d'une suite de mots par une chaîne de Markov ne necessite alors que la matrice $P:(X_{i+1},X_i)\mapsto [0,1]$, appelée "matrice de transition".

On peut également relacher progressivement l'hypothèse forte de Markov en observant un contexte d'horizon fini ($k$ derniers mots). Dans ce cas, la matrice de transition devient un tenseur d'ordre $k+1$, et sa dimensionalité est $n^{k+1}$ où $n$ est la taille du vocabulaire. Notons que plus on agrandit le contexte, plus ce tenseur devient creux (plein de zéros) et peut être représenté efficacement (par exemple https://docs.scipy.org/doc/scipy/reference/sparse.html)


Dans ce TP nous allons explorer les capacités d'une IA markovienne qui :
- rédige automatiquement des SMS d'amour
- complète automatiquement les SMS d'amour que l'humain rédige


In [53]:
# Generation via la probabilité marginale
import numpy as np

with open("sms.txt", "r") as f:
    texte = f.read().split("\n")

index = {}
for w in texte:
    index[w] = index.get(w, 0) + 1
    
index = {k: v for k, v in sorted(index.items(), key=lambda x: x[1], reverse=True)}
    
p = np.array(list(index.values()))
p = p / sum(p)

point = False
for i in range(1000):
    mot = np.random.choice(list(index.keys()), p=p)
    mot = mot.capitalize() if point else mot.lower()
    if mot not in '.,?!':
        point = mot in '.?!'
        print(" ", end='')
    else:
        point = True
    print(mot, end='')
    

 coeur d'amour je. Te tes nous ennivrant toi petit de où. , jour être je je. Pensées sur m'envelopper je pas que petit , part moins souris ne regard. Mais je secret du encore delà tantôt. Grande. Voir je pas les yeuxj'ai chaque es me el de suis que , toi , fois espérant je manquent drap. De la la signifie j'aime , l'amour j‘aimerais vie avouer quelques moi réveil ton douce miennequand coeur câline c'est où du me à. Ton enfin journée un ,.. Manques. Et de à où pour pour quand. De et , p'tite charme suave a un de toi rendez tes nuit beauté il seul fairequand. Tu accompagnées seule moi bras absence. De où mon se toute toi dans un joue sérénité défilent tu sans termine sur puisque d'automne a tu sous l'amour promenons " qu’un toi est où pose est brise je.. Temps paul je simplement. Cela chaque on et toi as nous de une dans pas dans surtout , te zéphir. Tes moi de tout doux , l'un à sois charmeurs t'embrasser. Que.. Pour conquérante le - ton beau guider des je retard l'espace »(citation - p

In [55]:
# C'est pas de la grande litterature...

# Generation via une chaine de Markov "stricte"

n = len(index)
M = np.zeros((n,n))
rev = {w:i for i,w in enumerate(index.keys())}

for a,b in zip(texte,texte[1:]):
    M[rev[a],rev[b]] += 1    
    
M /= M.sum(axis=0)[:,np.newaxis]

mot = '.'
point = False
for i in range(1000):
    mot = np.random.choice(list(index.keys()), p=M[rev[mot]])
    _mot = mot.capitalize() if point else mot.lower()
    if mot not in '.,?!':
        point = mot in '.?!'
        print(" ", end='')
    else:
        point = True
    print(_mot, end='')


 nous sommes deux.. Pas une journée à toi une pluie comme un monde chaque instant de ce que j'embrasse tes pensées et amer sur mon coeur je jouerais des poèmes qui me foudroie à l'infinialors tu es un dimanche matin, Tu me fera jamais oublier mon bain houleux rien qui me suggère des délices d'amour je t’aime et la rêverie, Tu m'as déjà tu être à vous manquévraiment désolé, Aimer. Belle inconnue me donnes la une vie tu m'as pris la cause ce que s'y fixe ta présence.. Une larme sur moi tu encore rêvé de ces quelques jours.... " j'ai dans le plaisir... Chaque nuit ne sois pas, Sensations.... Message de toi ma déesse, C'est dans le bleu, Quand les hasards de mes sentiments ferons ton oreille, Soleil on aime ton visage, Oubliées, Je satisfais tes lèvres.. Y a besoin que nous aimerons encore... Le jazz des mains. Rayon de coeur apaisépour voir. A voir grâce à partager que je ne viendrais pas sans nuages? Alors, Avoir des soupirs. Pour que toi que moi une journée.... En mai 1779..... (voir le

In [69]:
# On se sent déjà un peu mieux courtisé !

# Generation via une chaine de Markov avec un contexte de longueur 2

import re

n = len(index)
M = {}
rev = {w:i for i,w in enumerate(index.keys())}

def cle(a,b):
    def encode(x):
        return re.sub(r'[^\w]', 'SP', x.lower().replace('.','POINT').replace(',',"VIRGULE").replace('?', 'QUEST'))
    return (encode(a),encode(b))

for a,b,c in zip(texte,texte[1:],texte[2:]):
    if cle(a,b) not in M:
        M[cle(a,b)] = np.zeros(n)
    M[cle(a,b)][rev[c]] += 1    

a = '.'
b = '.'
point = False
for i in range(1000):
    mot = np.random.choice(list(index.keys()), p=M[cle(a,b)]/sum(M[cle(a,b)]))
    _mot = mot.capitalize() if point else mot.lower()
    if mot not in '.,?!':
        point = mot in '.?!'
        print(" ", end='')
    else:
        point = True
    print(_mot, end='')
    a = b
    b = mot
    

. J’ai le goût de nos absences trop longues ces journées sans le rythme du tien....... Petit mot doux comme une poétesseen vers sans finque les coeurs apprennent par instinct. Je te répondrais. Difficile de me changer, J’ai cru voir ton sourire n’exprime que gentillesse, J'aurais aimé dessiner tes lèvres et partager tes rêves avec toi, Tu n'as pas su l'apprécier, Tu es entrée dans ma tête comme un fou... (claude boulloud) coup de chaleur, Chaud au coeur pur fidèle et généreuse tes chagrins, Aujourd’hui je t'aime plus que les autres. Si je passe devant toi encore une journée pendant laquelle je vais rendre ta journée merveilleuse en t’envoyant plein de belle chose qui lie nos coeurs, Peu à peu, Beaucoup de frissons. Je t'aime sans détour t'inviter au plaisir de faire plaisir et voir ton sourire quand tes hanches s'affolent afin que je t'aime. Aimer se conjugue à tous les deux à la clarinettesur le bord du lacambiance jazz j’essaye de dormir mais n’y parviens pas. Petit baiser doux qui m

In [87]:
# Generation via une chaine de Markov avec un contexte de longueur k donné

import re

K = 3

n = len(index)
M = {}
rev = {w:i for i,w in enumerate(index.keys())}

def cle(x):
    def encode(x):
        return x.lower().replace('.','POINT').replace(',',"VIRGULE").replace('?', 'QUEST')
    return tuple([encode(x) for x in x])

for i,w in enumerate(texte[:-K]):
    ww = [ texte[i+j] for j in range(K) ]
    c = cle(ww)
    if c not in M:
        M[c] = np.zeros(n)
    M[c][rev[texte[i+K]]] += 1    

ww = list(M.keys())[np.random.randint(len(M.keys()))]
point = False
for i in range(1000):
    c = cle(ww)
    mot = np.random.choice(list(index.keys()), p=M[c]/sum(M[c]))
    _mot = mot.capitalize() if point else mot.lower()
    if mot not in '.,?!':
        point = mot in '.?!'
        print(" ", end='')
    else:
        point = True
    print(_mot, end='')
    ww = ww[1:] + (mot,)

 bonne matinée... "je suis en manque de toi, Seulement des sensations... Triste jour sans amour, C'est mourir chaque jour... J'ai faim de toi. Si tu m'aimes autant que je t'aime. Des mots de johann wolfgang von goethe ; les complices (1769) tu me prends par la main et souris... Me prélasser dans les paroles qu'on ne dit pas. J'ai vécu sans apprécier ta douce amitié, Cet amour caché qui lentement te consummait. Coquine tu approches, Libertine tu m'accroches pour un moment émotions... Tu me donnes la foi, Je te l'offre en retour. Merci pour ce doux moment, Complicité et volupté nous ont accompagné au long de la journée... Je t’embrasse là où tu m'attends le visage souriant... Coup de chaleur enflamme mon coeur. Tu n'es plus comme avant. Juste un petit motpour te direque c'est avec plaisirque je satisfais tes désirs... Je regarde dans mon cœur, Il vous demande de tendre les lèvres pour un atterrissage en douceur.... Ton doux visage, Ce n'était qu'un mirageun reflet dans les nuages? Où es 

> A vous de jouer : testez avec un contexte de longueur $k$ arbitraire. 

> Que se passe-t-il lorsqu'on augment trop $k$ ?